**CMPE-257 Machine Learning Homework: NLP Topic Modeling and Summarization for Poems**



Part 1: NLP POS Substitution for Poems

Find a couple of  poems from 2 poets [for example Robert Bly and Emily Dickinson ]

recommendation: scrape at least 10 poems by each of your 2 poets. ==> topic modeling requires more data to come up with sensible topics.

Home Work

1. Nice printing etc, see slides in Slack [edited to list details below]
    1. Print it out on the console so you can read and assist in evaluating the outcome of the transposition. Is the transposed poem a good poem? Does it make sense? Was the semantic similarity a good enough way of finding a suitable substitution?  
    2. Do I have to run the notebook again even though I have run it before and have produced results? If so, write defensive code to avoid re-running the parts of the notebook.
        1. scraped data for your poet's poems (10 each at least)
        2. json representation : parts of speech, author, poem name, poem text, url
        3. text files
    3. When I want to substitute/transpose the POS, Which POS should I use from the other poet?
        1. start with a poem from poet A
        2. find POS for poet A
        3. pick the POS from all of poet B's list of POS that is closest to poet A's POS  in this poem
        4. transpose it
        5. do this for all poems for poet A, then do this for all poems(initially just for two poems)

    4. Analyze the similarity of poems
        1. as-is between poets
        2. similarity when you swap nouns
        3. similarity when you swap verbs
        4. similarity when you swap adjectives
        5. similarity when you swap adverbs
        6. similarity when you swap all


In [1]:
poet1 = "Charles Bukowski"
poet2 = "Sylvia Plath"
store_loc = "/content/drive/MyDrive/Poets"

#Part 1 : POS Substitutions for Poetry

In [3]:
!pip install nltk
!pip install sentence-transformers
!pip install contractions
!pip install pandas
!pip install bertopic
!pip install gensim
!pip install pyLDAvis
!pip install spacy
!pip install requests
!pip install beautifulsoup4
!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 794.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing 

In [4]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [5]:
! pip install numpy

In [6]:
pip install numpy==1.26.4 --force-reinstall

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
import requests
import pandas as pd
import bs4 as bs
import urllib.request
import json
import contractions
import re
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from nltk.corpus import stopwords
from transformers import pipeline
from spacy import displacy
from bertopic import BERTopic
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pyLDAvis
import gensim
import spacy
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
from gensim.utils import simple_preprocess

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings('ignore', 'Calling')
warnings.filterwarnings('ignore', 'For a faster implementation')
warnings.filterwarnings('ignore', 'In')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')
warnings.filterwarnings('ignore', '`np')
warnings.filterwarnings('ignore', 'SelectableGroups')

In [4]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/

True

In [5]:
import os
import nltk

# Create a local directory for NLTK data
nltk_data_dir = os.path.expanduser('~/nltk_data')
os.makedirs(nltk_data_dir, exist_ok=True)

# Add to NLTK's path
nltk.data.path.append(nltk_data_dir)

# Download required resources
nltk.download('punkt', download_dir=nltk_data_dir)
nltk.download('averaged_perceptron_tagger', download_dir=nltk_data_dir)
nltk.download('wordnet', download_dir=nltk_data_dir)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Scrape 2 poems from 2 Poets

In [6]:
def clean_txt(text):
    text=text.replace(u'\xa0', u' ').replace(r'\r',u'\n').replace(r'\n', ' ').replace('\n', ' ').replace('\t', ' ')
    text = contractions.fix(text)
    text = text.strip().lower()
    text = re.sub(r"'", ' ', text)
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    text = ' '.join([w for w in text.split() if len(w)>1])
    text = re.sub(' +', ' ', text)
    text = ''.join(text)
    return text

def scrapePoemFromURL(poem_url_list):
    poems_results=[]
    for url in poem_url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        poem_for_human = soup.find_all('div', class_="poem_body")[0].text.replace("© by owner. provided at no charge for educational purposes", "")
        poem_for_machine = (clean_txt(poem_for_human))
        poem_title = soup.find_all('h1')[0].text
        poems_results.append([poem_title, poem_for_machine, poem_for_human])
    return poems_results

In [7]:
def get_poem_url_json(poet):
    site = "https://allpoetry.com"
    search = "https://allpoetry.com/items/read_by/"
    page = 1
    URL_list = list()
    while len(URL_list) < 10:
        url = search + poet + "?page=" + str(page)
        response = requests.get(url)
        soup = bs.BeautifulSoup(response.text, 'lxml')
        poem_list = soup.find_all('a', href=True, class_='nocolor fn')
        for i in range(len(poem_list)):
            #print(poem_list[i]['href'])
            URL_list.append(site + poem_list[i]['href'])
            if len(URL_list) == 10:
                break
        page += 1
    return {"poet":poet, "URL":URL_list}

In [8]:
pd_title_list = ["Title", "Poem", "Poem_for_Human"]

In [9]:
def getPoem(file_name, URLs):
    df = pd.DataFrame()
    if os.path.exists(file_name):
        df = pd.read_csv(file_name)
    else:
        os.makedirs(os.path.dirname(file_name), exist_ok=True)
        df = pd.DataFrame(scrapePoemFromURL(URLs), columns=pd_title_list)
        df.to_csv(file_name)
    return df

If the poem file exist, avoid re-scrape it. Else, scrape it.

In [10]:
file_name2 = store_loc + poet2.replace(" ", "_") + ".csv"
print(file_name2)
JWG_df = getPoem(file_name2, get_poem_url_json(poet2)['URL'])

/content/drive/MyDrive/PoetsSylvia_Plath.csv


In [11]:
file_name = store_loc + poet1.replace(" ", "_") + ".csv"
print(file_name)
WS_df = getPoem(file_name, get_poem_url_json(poet1)['URL'])

/content/drive/MyDrive/PoetsCharles_Bukowski.csv


## POS in all poem in JSON

In [12]:
def buildPoemArray(data):
    poet = {}
    poems = list()
    for i in data:
        idx = "poem"+str(i)
        poet[idx+"_title"] = data[i]["Title"]
        poet[idx] = data[i]["Poem"]
        poems.append(idx)
    poet['poems_array'] = poems
    return poet

In [13]:
def extractAllPOS(poet):
    global_vrb = set()
    global_nns = set()
    global_adj = set()
    for key in poet['poems_array']:
        text = poet[key]
        word_list = word_tokenize(text)
        pos_value = nltk.pos_tag(word_list)
        poet["pos_"+key] = pos_value

        vrb = set([word for (word, pos) in pos_value if (pos.startswith('VB'))])
        nns = set([word for (word, pos) in pos_value if (pos.startswith('NN'))])
        adj = set([word for (word, pos) in pos_value if (pos.startswith('JJ'))])

        poet["verbs_"+key] = list(vrb)
        poet["nouns_"+key] = list(nns)
        poet["adjectives_"+key] = list(adj)

        global_vrb = set.union(global_vrb, vrb)
        global_nns = set.union(global_nns, nns)
        global_adj = set.union(global_adj, adj)

    poet["all_verbs"] = list(global_vrb)
    poet["all_nouns"] = list(global_nns)
    poet["all_adjectives"] = list(global_adj)


In [14]:
def getPOS(file_name, df):
    pos = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos = json.load(file)
    else:
        pos = buildPoemArray(json.loads(df.to_json(orient="index")))
        extractAllPOS(pos)
        with open(file_name, 'w') as file:
            json.dump(pos, file)
    return pos

In [15]:
def getJSON(file_name, df_name_1, df_1, df_name_2, df_2):
    pos_json = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            pos_json = json.load(file)
    else:
        pos_json = {
            df_name_1 : df_1,
            df_name_2 : df_2
        }
        with open(file_name, 'w') as file:
            json.dump(pos_json, file)
    return pos_json

If the POS JSON file exist, avoid re-POS it. Else, POS it.

In [16]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [17]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [18]:
file_name = store_loc + poet1.replace(" ", "_") + ".json"
WS_pos = getPOS(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + ".json"
JWG_pos = getPOS(file_name, JWG_df)
file_name = store_loc + poet1.replace(" ", "_") + "-" + poet2.replace(" ", "_") + ".json"
poets_json = getJSON(file_name, poet1.replace(" ", "_"), WS_pos, poet2.replace(" ", "_"), JWG_pos)

## Transpose verbs, Adjectives and nouns

In [19]:
def getSemanticSimilarity(p1, p2, model):
    embedding1 = model.encode(p1, convert_to_tensor=True)
    embedding2 = model.encode(p2, convert_to_tensor=True)
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [20]:
def getSemanticSimilarityEmbedding(embedding1, embedding2, model):
    return util.pytorch_cos_sim(embedding1, embedding2).item()

In [21]:
def get_poet_embedding_dict(poet, model):
    poet_dict = dict()
    for word in poet:
      if word not in poet_dict:
        poet_dict[word] = model.encode(word, convert_to_tensor=True)
    return poet_dict

In [22]:
def swapWords(poet1, poet2, poem):
    model = SentenceTransformer('stsb-roberta-large')
    import time
    start = time.time()
    poet1_dict = get_poet_embedding_dict(poet1, model)
    end = time.time()
    print(end - start)
    start = time.time()
    poet2_dict = get_poet_embedding_dict(poet2, model)
    end = time.time()
    print(end - start)

    cs_threshold = 0.4
    new_poem = poem
    for word1 in poet1:
        max_word = ""
        max_sim = 0
        for word2 in poet2:
            if word1 == word2:
                continue
            sim = getSemanticSimilarityEmbedding(poet1_dict[word1], poet2_dict[word2], model)
            if max_sim < sim:
                max_sim = sim
                max_word = word2
        if max_sim > cs_threshold:
            new_poem = new_poem.replace(word1, max_word)
    return new_poem

## replace poet A POS with poet B POS based on semantic similarity

In [23]:
def newPoem(file_name, poet_1_pos, poet_2_pos, poem, index):
    new_poem = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            new_poem = file.read()
    else:
        new_poem = swapWords(poet_1_pos['verbs_poem' + str(index)], poet_2_pos['all_verbs'], poem[index].lower())
        new_poem = swapWords(poet_1_pos['adjectives_poem'+ str(index)], poet_2_pos['all_adjectives'], new_poem)
        new_poem = swapWords(poet_1_pos['nouns_poem'+ str(index)], poet_2_pos['all_nouns'], new_poem)
        with open(file_name, 'w') as file:
            file.write(new_poem)
    return new_poem

If the new poem text file exist, avoid replace it. Else, replace it.

In [24]:
for i in range(0):
    file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-' + WS_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, i)

In [25]:
for i in range(0):
    file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-' + JWG_df.Title[i].replace('\'','').replace('-','').replace(',','').replace(':','').replace('  ', ' ').replace(' ', '_') + '.txt'
    newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, i)

In [26]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem0.txt'
WS_p1_new = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 0)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

28.058290243148804
255.7022466659546
7.418270111083984
171.1962592601776
20.92044448852539
447.6041648387909


In [27]:
file_name = store_loc + poet1.replace(" ", "_") + '-' + poet2.replace(" ", "_") + '-poem1.txt'
WS_p2_new = newPoem(file_name, WS_pos, JWG_pos, WS_df.Poem_for_Human, 1)

31.042887449264526
270.69949674606323
21.239264249801636
175.20398807525635
51.45262360572815
431.40199995040894


In [29]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem0.txt'
JWG_p1_new = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 0)

20.9882755279541
197.6982638835907
11.619175434112549
95.9996554851532
25.318678855895996
285.8028314113617


In [30]:
file_name = store_loc + poet2.replace(" ", "_") + '-' + poet1.replace(" ", "_") + '-poem1.txt'
JWG_p2_new = newPoem(file_name, JWG_pos, WS_pos, JWG_df.Poem_for_Human, 1)

44.17049765586853
213.2005157470703
28.917711973190308
95.3032214641571
79.52766680717468
279.30065631866455


## Similarity of poems

Before Swapping: Poem 1

In [31]:
getSemanticSimilarity(WS_df.Poem[0], JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

0.4638702869415283



```
# This is formatted as code
```

After Swapping: Poem 1

In [32]:
getSemanticSimilarity(WS_p1_new, JWG_df.Poem[0], SentenceTransformer('stsb-roberta-large'))

0.409005731344223

Before Swapping: Poem 2

In [33]:
getSemanticSimilarity(WS_df.Poem[1], JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

0.4651166498661041

After Swapping: Poem 2

In [34]:
getSemanticSimilarity(WS_p2_new, JWG_df.Poem[1], SentenceTransformer('stsb-roberta-large'))

0.4816325902938843

**Compare** with old and new: Poem 1

In [35]:
getSemanticSimilarity(WS_df.Poem[0], WS_p1_new, SentenceTransformer('stsb-roberta-large'))

0.7783671021461487

Compare with old and new: Poem 2

> Add blockquote



In [36]:
getSemanticSimilarity(WS_df.Poem[1], WS_p2_new, SentenceTransformer('stsb-roberta-large'))

0.6107051968574524

## In human readable format

In [37]:
print(WS_p1_new)


	there's a blue in my heart—— that
	wanteds to given out
	but i'm too dangerous for him,
	i said, stuck in there, i'm not go
	to go want look
	you.
	there's a blue in my heart—— that
	wanteds to given out
	but i pours whwerekey on him and breathe
	cigwerette candle
	and the salty and the bars
	and the cargo crunching
	never knew that
	he's
	in there.

	there's a blue in my heart—— that
	wanteds to given out
	but i'm too dangerous for him,
	i said,
	stuck doingwn, doing you wanted to stir
	me up?
	you wanted to stir up the
	works?
	you wanted to buzzed my books case in
	elements?
	there's a blue in my heart—— that
	wanteds to given out
	but i'm too great, i only go him out
	at darkness sometimes
	when everything's resting.
	i said, i knew that you're there,
	so doingn't been
	terrible.
	then i pours him back,
	but he's singing a small
	in there, i hasn't quite go him
	dying
	and we resting togivenher like
	that
	with our
	strange accomplishment
	and it's beautiful enough to
	made a men

In [38]:
print(WS_p2_new)

too much 
too small 

too heavy
too small
or empty. 

smiles or
cry 

malignity
lovingrs 

absence with effaced like
the back of
thins ticking 

wars run through
town of vampire
wave wine
bayoneting and hurt
vampire. 

or an last man in a poor bed
with a picture of m. motes. 

there were a linen in thwere everything so happy
that you can eyeing it in the calm moves of
the hand of a time.

picture so sleep
obscene
either by loving or no loving. 

picture just were not great to each more
one on one. 

the valuable were not great to the valuable
the terrible were not great to the terrible. 

we were dangerous. 

our elect show tell us
that we can all been
large-associations accomplishment.

it hasn't tell us
about the gull
or the deaths. 

or the torture of one thing
ach in one piece
alone 

blank
blank to 

simmering a garden. 

picture were not great to each more.
picture were not great to each more.
picture were not great to each more. 

i guess they never will been.
i doingn't asks th

In [39]:
print(JWG_p1_new)


	i yawned silver and everything. i has no expectation.
	whatever i look i dull immediately
	just as it was, unmwasted by lovers or dwaslike.
	i yawned not splendid, only knows
	the eyes of a few blessed, four-crossed.
	much of the times i rest on the different walls.
	it was pale, with litter. i has looking at it so long
	i thinking it was thing of my love. but it flies.
	look and night gone us over and over.
	now i yawned a lake. a women stems over me,
	looking my look for what she really was.
	then she turn to those laughs, the cigar or the night.
	i look her backs, and look it faithfully.
	she gets me with sadness and an hordes of held.
	i yawned big to her. she came and went.
	each hey it was her front that rewrite the night.
	in me she has dying a little girls, and in me an long women
	climbing toward her days after days, like a bad fwash.     system analyswas


In [40]:
print(JWG_p2_new)


	i has end it again.
	  one years in every doingg
	  i man itfronts
	 
	  a right of walk avenues, my thin
	  btrue as a godsen lyawnedpshase,
	  my true feet
	 
	  a write,
	  my front a ftimeseureless, sure
	  hey loneliness.
	 
	  went off the napkin
	  0 my hcream.
	  doing i madness?fronts
	 
	  the nose, the eyes punched, the much time of toothless?
	  the sad brtimeseh
	  will vanwereh in a days.
	 
	  soon, soon the soft
	  the death bars cream will been
	  at landlords on me
	 
	  and i a witty women.
	  i yawned only thirty.
	  and like the cat i has nine times to die.
	 
	  thwere were numbeenr three.
	  what a trwealth
	  to die each decades.
	 
	  what a million filyawnedents.
	  the whiskey-crushed many
	  move in to look
	 
	  them toweling me held and feet
	  the fat balls chance.
	  gentleman, ladies
	 
	  these were my helds
	  my floor.
	  i may been thin and backbone,
	 
	  nevertheless, i yawned the syawnede, same women.
	  the fresh times it happens i were doingg

##Extra Credit - Do two distillations including: POS, embeddings POS; do embeddings to find which poets are closest to each other in style

In [41]:
!gdown --id 1Z8M_4_LKNd6iMogJJ_I9G-6_wcdrZBld -O Charles Bukowski.json

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY]
             [--speed SPEED] [--no-cookies] [--no-check-certificate]
             [--continue] [--folder] [--remaining-ok] [--format FORMAT]
             [--user-agent USER_AGENT]
             url_or_id
gdown: error: unrecognized arguments: Bukowski.json


In [42]:
!gdown --id 16N24od__tcRdlLI2SvtLWk1ThKSAAzff -O Sylvia Plath.json

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY]
             [--speed SPEED] [--no-cookies] [--no-check-certificate]
             [--continue] [--folder] [--remaining-ok] [--format FORMAT]
             [--user-agent USER_AGENT]
             url_or_id
gdown: error: unrecognized arguments: Plath.json


In [43]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 43.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [44]:
import json
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load spaCy model with vectors
nlp = spacy.load("en_core_web_md")

# Load POS-rich JSON files
with open('/content/Charles Bukowski.json') as f:
    gibran = json.load(f)
with open('/content/Sylvia Plath.json') as f:
    whitman = json.load(f)

# Extract POS word lists
g_nouns = gibran["all_nouns"]
g_verbs = gibran["all_verbs"]
g_adjs  = gibran["all_adjectives"]

w_nouns = whitman["all_nouns"]
w_verbs = whitman["all_verbs"]
w_adjs  = whitman["all_adjectives"]

# Function to get average embedding for a list of words
def average_vector(words):
    vectors = [nlp(word).vector for word in words if nlp(word).has_vector]
    if not vectors:
        return np.zeros(nlp.vocab.vectors_length)
    return np.mean(vectors, axis=0)

# Get average POS embeddings for each poet
g_noun_vec = average_vector(g_nouns)
g_verb_vec = average_vector(g_verbs)
g_adj_vec  = average_vector(g_adjs)

w_noun_vec = average_vector(w_nouns)
w_verb_vec = average_vector(w_verbs)
w_adj_vec  = average_vector(w_adjs)

# Cosine similarity between poets' POS embeddings
def sim(vec1, vec2):
    return float(cosine_similarity([vec1], [vec2])[0][0])

# Compute similarities
similarity_report = {
    "noun_similarity": sim(g_noun_vec, w_noun_vec),
    "verb_similarity": sim(g_verb_vec, w_verb_vec),
    "adjective_similarity": sim(g_adj_vec, w_adj_vec)
}

# Show results
print("🔍 Style Similarity Between Charles Bukowski and Sylvia Plath:")
for k, v in similarity_report.items():
    print(f"{k.replace('_', ' ').title()}: {v:.4f}")

# Save to a new file for submission
with open('/content/poet_style_similarity_v2.json', 'w') as f:
    json.dump(similarity_report, f, indent=2)

FileNotFoundError: [Errno 2] No such file or directory: '/content/Charles Bukowski.json'

# Part 2 : Topic Modeling and Summarization

## Summarization

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

In [ ]:
def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data
def SummaryOfPoem(poet):
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key]
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=50, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text

  return poet

In [ ]:
def getSummary(file_name, df):
    summary = ""
    if os.path.exists(file_name):
        with open(file_name, 'r') as file:
            summary = json.load(file)
    else:
        summary = SummaryOfPoem(buildPoemArray(json.loads(df.to_json(orient="index"))))
        with open(file_name, 'w') as file:
            json.dump(summary, file)
    return summary

In [ ]:
file_name = store_loc + poet1.replace(" ", "_") + "-Summary.json"
WS_summary = getSummary(file_name, WS_df)
file_name = store_loc + poet2.replace(" ", "_") + "-Summary.json"
JWG_summary = getSummary(file_name, JWG_df)

In [ ]:
WS_summary

## Topic Modeling

## LDA

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
def choherence_perplexity_calculator(poem_list):
  data_words = list(sent_to_words(poem_list))
  print(data_words[:1])
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  print(trigram_mod[bigram_mod[data_words[0]]])


  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent))
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])

  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  #return vis
  return lda_model, corpus, id2word

In [ ]:
WS_input = json.loads(WS_df.to_json(orient="index"))
JWG_input = json.loads(JWG_df.to_json(orient="index"))

In [ ]:
poem_count = len(WS_input)
WS_list=[]
for j in WS_input:
  poem_index =  str(j)
  WS_input[poem_index].pop('Poem_for_Human')
  WS_list.append(WS_input[poem_index])
print(WS_list)

In [ ]:
poem_count = len(JWG_input)
JWG_list=[]
for j in JWG_input:
  poem_index =  str(j)
  JWG_input[poem_index].pop('Poem_for_Human')
  JWG_list.append(JWG_input[poem_index])
print(JWG_list)

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(WS_list)

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(JWG_list)

In [ ]:
gensimvis.prepare(lda_model, bow, diction)

In [ ]:
def ner(text):
  return spacy.load("en_core_web_sm")(text)

In [ ]:
original = ner(WS_input['0']['Poem'])

In [ ]:
displacy.render(original, style="ent", jupyter=True)



```
# This is formatted as code
```

## BERTopic

In [ ]:
def remove_stopwords(texts):
    stopword_list = nltk.corpus.stopwords.words('english')
    return [[word for word in simple_preprocess(str(doc)) if word not in stopword_list] for doc in texts]

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(WS_df.Poem.to_list() + JWG_df.Poem.to_list()))

print(data_words[:1])
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
docs = WS_df.Poem.to_list() + JWG_df.Poem.to_list()

In [ ]:
docs = remove_stopwords(docs)
docs = make_bigrams(docs)
docs = make_trigrams(docs)
docs = lemmatization(docs)

In [ ]:
docs=[[' '.join(i)] for i in docs]
newDocs = list()
for i in docs:
    for j in i:
        newDocs.append(j)
print(newDocs)

In [ ]:
model = BERTopic(embedding_model=SentenceTransformer('stsb-roberta-large'))

topics, probabilities = model.fit_transform(newDocs)

In [ ]:
model.get_topic_freq().head(11)

In [ ]:
model.get_topic(-1)

## News scraping and Topic Modeling

In [ ]:
def scrapeNewsFromURL(url_list):
    news_results=[]
    for url in url_list:
        header = {'User-Agent':'Mozilla/5.0'}
        requests = urllib.request.Request(url,headers=header)
        requestread = urllib.request.urlopen(requests).read()
        soup = bs.BeautifulSoup(requestread,'html.parser')
        raw = soup.find_all('div', class_="l-container")[0].text
        clean = (clean_txt(raw))
        news_results.append(clean)
    return news_results

In [ ]:
NEWS_URLs = [
"https://www.cnn.com/2022/04/16/asia/north-korea-weapon-test-intl-hnk/index.html",
"https://www.cnn.com/2022/04/18/europe/ukraine-russia-mariupol-monday-intl/index.html",
"https://www.cnn.com/2022/04/08/americas/brazil-amazon-deforestation-latam-intl/index.html",
"https://www.cnn.com/2022/04/07/americas/colombia-landslide-miners-intl/index.html"
]

In [ ]:
NEWS_list=scrapeNewsFromURL(NEWS_URLs)

In [ ]:
lda_model, bow, diction = choherence_perplexity_calculator(NEWS_list)

In [ ]:
gensimvis.prepare(lda_model, bow, diction)